In [31]:
import pandas as pd
import sqlite3
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import csv

# Import XGBoost classifier
from xgboost import XGBClassifier

# Import scikit-learn functions
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Import scikit-plot functions
from scikitplot.metrics import plot_roc_curve
from scikitplot.metrics import plot_precision_recall_curve
from scikitplot.metrics import plot_calibration_curve

# Import SciPy function
from scipy.spatial import distance

pd.set_option("display.max_columns", None)


In [32]:
# Data Directory
data_dir = "../../Data/Big-Data-Cup-2021"
bucketless_data_dir = '../../Data/bdc/data'
tracking_data_dir = "{}/TrackingData".format(data_dir)
db_path = '/Users/keltim01/Documents/databases/'

#connect to database 
conn = sqlite3.connect(db_path + 'bdc_2022.db')
cursor = conn.cursor()

In [33]:
df_train = pd.read_sql(
    '''
    SELECT * FROM pbp_training 
    WHERE (is_shot = 1 OR is_goal = 1)
    '''
,conn)

In [34]:
df_test = pd.read_sql(
    '''
    SELECT * FROM play_by_play
    WHERE is_shot = 1
    '''
,conn)

## Choosing Features of the Expected Goals Model
### Source:
source/inspiration: https://hockey-graphs.com/2019/08/12/expected-goals-model-with-pre-shot-movement-part-1-the-model/

- Unblocked Shots (Fenwick Shots)


### Features:
- shot type, 
- shot location 
- distance from the net
- angle from the net
- type event prior 
- location event prior
- time 
- distance from last event
- time since last event
- angle change between the shot and its prior event
- score state
- strength state

- Dataset mostly contains neutral site games so the home and away should not be as important

In [35]:
df_test.columns

Index(['index', 'game_date', 'season_year', 'team_name', 'opp_team_name',
       'venue', 'period', 'clock_seconds', 'situation_type', 'goals_for',
       'goals_against', 'player_name', 'event', 'event_successful', 'x_coord',
       'y_coord', 'event_type', 'player_name_2', 'x_coord_2', 'y_coord_2',
       'event_detail_1', 'event_detail_2', 'event_detail_3', 'frame_id_1',
       'frame_id_2', 'home_team', 'away_team', 'game_id', 'is_shot', 'is_goal',
       'event_id', 'team_id', 'player_id', 'detail_1_code', 'goal_diff',
       'game_seconds_remaining', 'event_code', 'event_type_code',
       'skaters_for', 'skaters_against', 'strength_state', 'distance_to_goal',
       'angle_to_goal', 'prev_event', 'prev_event_code', 'prev_event_type',
       'prev_event_type_code', 'prev_event_x_coord', 'prev_event_y_coord',
       'prev_event_game_seconds_remaining', 'prev_event_distance_to_goal',
       'prev_event_angle_to_goal', 'time_diff_last_event',
       'angle_diff_last_event', 'distanc

In [36]:
df_train.columns

Index(['index', 'game_date', 'home_team', 'away_team', 'period', 'clock',
       'home_team_skaters', 'away_team_skaters', 'home_team_goals',
       'away_team_goals', 'team', 'player', 'event', 'x_coord', 'y_coord',
       'detail_1', 'detail_2', 'detail_3', 'detail_4', 'player_2',
       'x_coordinate_2', 'y_coordinate_2', 'game_id', 'event_id', 'team_id',
       'player_id', 'is_shot', 'is_goal', 'detail_1_code', 'detail_2_code',
       'detail_3_code', 'detail_4_code', 'is_home', 'goal_diff',
       'game_seconds_remaining', 'event_code', 'strength_state',
       'distance_to_goal', 'angle_to_goal', 'prev_event', 'prev_event_code',
       'prev_event_type', 'prev_event_type_code', 'prev_event_x_coord',
       'prev_event_y_coord', 'prev_event_game_seconds_remaining',
       'prev_event_distance_to_goal', 'prev_event_angle_to_goal',
       'time_diff_last_event', 'angle_diff_last_event',
       'distance_diff_last_event', 'second_prev_event',
       'second_prev_event_code', 'second

In [37]:
df_train['detail_1'].unique(), df_train['detail_2'].unique()

(array(['Snapshot', 'Fan', 'Slapshot', 'Wristshot', 'Deflection',
        'Wrap Around'], dtype=object),
 array(['On Net', 'Missed', 'Blocked'], dtype=object))

In [38]:
df_test['event_type'].unique(), df_test['event_detail_1'].unique()

(array(['Slapshot', 'Snapshot', 'Fan', 'Wristshot', 'Deflection',
        'Wrap Around'], dtype=object),
 array(['Blocked', 'Missed', 'On Net'], dtype=object))

In [39]:
df_test.head()

,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event
0,12,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1174,5 on 5,0,0,Jocelyne Larocque,Shot,0,149,13,Slapshot,None,None,None,Blocked,1,0,NaN,None,Olympic (Women) - United States,Olympic (Women) - Canada,5,1,0,5,0,42,0,0,3574.0,5,19,5,5,0,50.509900,1.389831,Play,3.0,Direct,6.0,153.0,76.0,3576.0,49.912423,1.104478,-2.0,0.285353,0.597477,Puck Recovery,4.0,None,-1.0,170.0,79.0,3577.0,41.620308,0.547945,-1.0,0.556532,8.292116
1,18,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1165,5 on 5,0,0,Sarah Fillier,Shot,0,148,4,Snapshot,None,None,None,Missed,1,0,NaN,None,Olympic (Women) - United States,Olympic (Women) - Canada,5,1,0,5,0,87,1,0,3565.0,5,21,5,5,0,56.975872,1.090909,Puck Recovery,4.0,None,-1.0,146.0,8.0,3565.0,55.912879,1.275362,0.0,-0.184453,1.062993,Play,3.0,Direct,6.0,130.0,67.0,3567.0,64.809336,2.448980,-2.0,-1.173617,-8.896457
2,58,8/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Canada,home,1,1095,5 on 5,0,0,Abby Roque,Shot,0,177,57,Snapshot,None,None,None,On Net,0,0,NaN,None,Olympic (Women) - United States,Olympic (Women) - Canada,5,1,0,5,4,1,2,0,3495.0,5,21,5,5,0,19.474342,0.896552,Puck Recovery,4.0,None,-1.0,150.0,69.0,3499.0,47.981767,1.509434,-4.0,-0.612882,-28.507425,Puck Recovery,4.0,None,-1.0,10.0,9.0,3502.0,183.090824,5.373134,-3.0,-3.863700,-135.109057
3,68,8/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Canada,home,1,1083,5 on 5,0,0,Kelly Pannek,Shot,0,163,32,Snapshot,None,None,None,On Net,0,0,NaN,None,Olympic (Women) - United States,Olympic (Women) - Canada,5,1,0,5,4,46,2,0,3483.0,5,21,5,5,0,28.969812,2.571429,Puck Recovery,4.0,None,-1.0,163.0,32.0,3483.0,28.969812,2.571429,0.0,0.000000,0.000000,Play,3.0,Direct,6.0,134.0,3.0,3485.0,68.529191,1.417722,-2.0,1.153707,-39.559379
4,82,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1061,5 on 5,0,0,Sarah Nurse,Shot,0,154,2,Snapshot,None,None,None,Missed,0,0,NaN,None,Olympic (Women) - United States,Olympic (Women) - Canada,5,1,0,5,0,89,1,0,3461.0,5,21,5,5,0,54.187176,0.888889,Puck Recovery,4.0,None,-1.0,151.0,5.0,3462.0,54.104066,1.040000,-1.0,-0.151111,0.083109,Dump In/Out,0.0,None,-1.0,35.0,82.0,3464.0,159.953900,3.924051,-2.0,-2.884051,-105.849833


In [40]:
df_test_model = pd.read_sql(
    '''
    SELECT player_name, is_goal, event_type_code, x_coord, y_coord, distance_to_goal, angle_to_goal, prev_event_code, prev_event_x_coord, prev_event_y_coord, second_prev_event_code, second_prev_event_x_coord, second_prev_event_y_coord, game_seconds_remaining, time_diff_last_event, angle_diff_last_event, distance_diff_last_event, prev_time_diff_last_event, prev_angle_diff_last_event, prev_distance_diff_last_event, goal_diff, strength_state
    FROM play_by_play
    WHERE is_shot = 1
    '''
,conn)

In [41]:
df_test['event_detail_1'].unique()

array(['Blocked', 'Missed', 'On Net'], dtype=object)

In [42]:
df_test_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   player_name                    709 non-null    object 
 1   is_goal                        709 non-null    int64  
 2   event_type_code                709 non-null    int64  
 3   x_coord                        709 non-null    int64  
 4   y_coord                        709 non-null    int64  
 5   distance_to_goal               709 non-null    float64
 6   angle_to_goal                  709 non-null    float64
 7   prev_event_code                709 non-null    float64
 8   prev_event_x_coord             709 non-null    float64
 9   prev_event_y_coord             709 non-null    float64
 10  second_prev_event_code         709 non-null    float64
 11  second_prev_event_x_coord      709 non-null    float64
 12  second_prev_event_y_coord      709 non-null    flo

In [43]:
df_test_model.shape

(709, 22)

In [44]:
df_train_model = pd.read_sql(
    '''
    SELECT is_goal, detail_1_code event_type_code, x_coord, y_coord, distance_to_goal, angle_to_goal, prev_event_code, prev_event_x_coord, prev_event_y_coord, second_prev_event_code, second_prev_event_x_coord, second_prev_event_y_coord, game_seconds_remaining, time_diff_last_event, angle_diff_last_event, distance_diff_last_event, prev_time_diff_last_event, prev_angle_diff_last_event, prev_distance_diff_last_event, goal_diff, strength_state
    FROM pbp_training 
    WHERE (is_shot = 1 OR is_goal = 1)
    '''
,conn)

In [45]:
df_train_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3656 entries, 0 to 3655
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   is_goal                        3656 non-null   int64  
 1   event_type_code                3656 non-null   int64  
 2   x_coord                        3656 non-null   int64  
 3   y_coord                        3656 non-null   int64  
 4   distance_to_goal               3656 non-null   float64
 5   angle_to_goal                  3656 non-null   float64
 6   prev_event_code                3656 non-null   float64
 7   prev_event_x_coord             3656 non-null   float64
 8   prev_event_y_coord             3656 non-null   float64
 9   second_prev_event_code         3656 non-null   float64
 10  second_prev_event_x_coord      3656 non-null   float64
 11  second_prev_event_y_coord      3656 non-null   float64
 12  game_seconds_remaining         3656 non-null   i

In [46]:
df_train_model.shape, df_test_model.shape

((3656, 21), (709, 22))

In [47]:
df_test_model['event_type_code'].unique()

array([19, 21,  8, 26,  5, 25])

In [48]:
features = ['event_type_code', 'x_coord', 'y_coord', 'distance_to_goal', 'angle_to_goal', 'prev_event_code', 'prev_event_x_coord', 'prev_event_y_coord', 'second_prev_event_code', 'second_prev_event_x_coord', 'second_prev_event_y_coord', 'game_seconds_remaining', 'time_diff_last_event', 'angle_diff_last_event', 'distance_diff_last_event','prev_time_diff_last_event', 'prev_angle_diff_last_event', 'prev_distance_diff_last_event', 'goal_diff', 'strength_state']
label = ['is_goal']
X_train = df_train_model[features]
y_train = df_train_model[label]
X_test = df_test_model[features]
y_test = df_test_model[label]

categorical_features_indices = np.where(X_train.dtypes != np.float)[0]

<ipython-input-48-da582c0a6357>:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(X_train.dtypes != np.float)[0]


In [49]:
parameters = {
    'nthread': [4],
    'objective': ['binary:logistic'],
    'max_depth': [3,4,5,6],
    'learning_rate': [0.01],
    'n_estimators': [100, 500, 1000],
    'seed': [42]
}

In [50]:
df_xg_model = pd.DataFrame()
auc_roc = []
kf = KFold(10, shuffle=True)
vals_y_test = pd.DataFrame()
vals_y_pred = np.ndarray(shape=(0,))

for train_idx, text_idx in kf.split(df_test_model):
    df_train_data = df_train_model.append(df_test_model.iloc[train_idx].copy(), ignore_index=True) 
    df_test_data = df_test_model.iloc[text_idx].copy()
    X_train = df_train_data[features]
    y_train = df_train_data[label]
    X_test = df_test_data[features]
    y_test = df_test_data[label]

    classifier = XGBClassifier()
    classifier = GridSearchCV(classifier, parameters, cv=5, scoring='roc_auc', verbose=2)
    classifier.fit(X_train, y_train)
    dfs_predictions = {}
    y_pred = classifier.predict_proba(X_test)
    dfs_predictions[label[0]] = pd.Series(y_pred[:, 1],index=df_test_data.index)
    df_predictions = pd.concat(dfs_predictions, axis=1)
    df_xg_model = df_xg_model.append(df_predictions)

    vals_y_test = vals_y_test.append(y_test) 
    vals_y_pred = np.concatenate([vals_y_pred,y_pred[:, 1]], axis=0)
    auc_roc.append(roc_auc_score(y_test, y_pred[:, 1]))

<ipython-input-50-86c9de49f390>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train_data = df_train_model.append(df_test_model.iloc[train_idx].copy(), ignore_index=True)
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-p

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[14:42:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, nthread=4, objective=binary:logistic, seed=42; total time=   0.2s
[14:42:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

...

/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=6, n_estimators=1000, nthread=4, objective=binary:logistic, seed=42; total time=   2.9s
[14:54:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=6, n_estimators=1000, nthread=4, objective=binary:logistic, seed=42; total time=   3.0s
[14:54:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


<ipython-input-50-86c9de49f390>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_xg_model = df_xg_model.append(df_predictions)
<ipython-input-50-86c9de49f390>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vals_y_test = vals_y_test.append(y_test)


In [51]:
new_auc_roc = roc_auc_score(vals_y_test, vals_y_pred)
new_auc_roc

0.8107302533532041

In [52]:
df_test_model['expected_goals'] = df_xg_model['is_goal']

In [53]:
df_test_model.columns

Index(['player_name', 'is_goal', 'event_type_code', 'x_coord', 'y_coord',
       'distance_to_goal', 'angle_to_goal', 'prev_event_code',
       'prev_event_x_coord', 'prev_event_y_coord', 'second_prev_event_code',
       'second_prev_event_x_coord', 'second_prev_event_y_coord',
       'game_seconds_remaining', 'time_diff_last_event',
       'angle_diff_last_event', 'distance_diff_last_event',
       'prev_time_diff_last_event', 'prev_angle_diff_last_event',
       'prev_distance_diff_last_event', 'goal_diff', 'strength_state',
       'expected_goals'],
      dtype='object')

In [54]:
df_test_model['is_goal'].sum(), df_test_model['expected_goals'].sum()

(38, 31.605682)

In [55]:
df_xg_model

,is_goal
11,0.007352
21,0.158927
30,0.061534
33,0.024476
51,0.005131
...,...
664,0.009982
672,0.095962
674,0.019071
685,0.020928


In [56]:
df_shots = pd.read_sql(
    '''
    SELECT * FROM play_by_play
    WHERE is_shot = 1
    '''
,conn)

In [57]:
df_shots['expected_goals'] = df_xg_model['is_goal']

In [58]:
df_shots.columns

Index(['index', 'game_date', 'season_year', 'team_name', 'opp_team_name',
       'venue', 'period', 'clock_seconds', 'situation_type', 'goals_for',
       'goals_against', 'player_name', 'event', 'event_successful', 'x_coord',
       'y_coord', 'event_type', 'player_name_2', 'x_coord_2', 'y_coord_2',
       'event_detail_1', 'event_detail_2', 'event_detail_3', 'frame_id_1',
       'frame_id_2', 'home_team', 'away_team', 'game_id', 'is_shot', 'is_goal',
       'event_id', 'team_id', 'player_id', 'detail_1_code', 'goal_diff',
       'game_seconds_remaining', 'event_code', 'event_type_code',
       'skaters_for', 'skaters_against', 'strength_state', 'distance_to_goal',
       'angle_to_goal', 'prev_event', 'prev_event_code', 'prev_event_type',
       'prev_event_type_code', 'prev_event_x_coord', 'prev_event_y_coord',
       'prev_event_game_seconds_remaining', 'prev_event_distance_to_goal',
       'prev_event_angle_to_goal', 'time_diff_last_event',
       'angle_diff_last_event', 'distanc

In [59]:
df_shots.to_sql('pbp_shots', conn, if_exists='replace')

709

In [60]:
df_shots['expected_goals'].describe()

count    709.000000
mean       0.044578
std        0.061186
min        0.000815
25%        0.010233
50%        0.020654
75%        0.058270
max        0.648573
Name: expected_goals, dtype: float64

In [61]:
df_shots.loc[df_shots['player_name'] == 'Michelle Karvinen',['expected_goals', 'is_goal','player_name']].sort_values(by='expected_goals', ascending=False)

,expected_goals,is_goal,player_name
161,0.606327,1,Michelle Karvinen
126,0.125761,0,Michelle Karvinen
513,0.119208,0,Michelle Karvinen
708,0.092369,1,Michelle Karvinen
625,0.084938,0,Michelle Karvinen
212,0.079089,0,Michelle Karvinen
140,0.073948,0,Michelle Karvinen
514,0.062669,0,Michelle Karvinen
657,0.058270,0,Michelle Karvinen
154,0.052989,0,Michelle Karvinen


In [62]:
df_shots.loc[(df_shots['player_name'] == 'Alina Muller') & (df_shots['strength_state'] > 0)]

,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,expected_goals
321,4996,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,737,5 on 4,2,1,Alina Muller,Shot,0,141,26,Wristshot,None,None,None,Blocked,1,0,219.0,None,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,0,1,737.0,5,26,5,4,1,51.703482,2.969697,Play,3.0,Direct,6.0,132.0,42.0,739.0,58.002155,116.000000,-2.0,-113.030303,-6.298674,Play,3.0,Direct,6.0,150.0,5.0,741.0,54.829280,1.066667,-2.0,114.933333,3.172875,0.013639
327,5049,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,642,5 on 4,2,1,Alina Muller,Shot,0,160,27,Slapshot,None,None,None,On Net,0,0,4887.0,None,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,2,1,642.0,5,19,5,4,1,33.767588,1.935484,Play,3.0,Direct,6.0,162.0,63.0,643.0,34.702305,1.365854,-1.0,0.569630,-0.934717,Play,3.0,Direct,6.0,134.0,38.0,645.0,56.180513,12.444444,-2.0,-11.078591,-21.478207,0.018059
687,10165,16/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Finland,away,3,960,5 on 4,0,2,Alina Muller,Shot,0,149,20,Wristshot,None,None,None,Blocked,1,0,3642.0,None,Olympic (Women) - Finland,Olympic (Women) - Switzerland,3,1,0,5,3,6,0,-2,960.0,5,26,5,4,1,46.768045,1.822222,Zone Entry,7.0,Carried,2.0,126.0,16.0,961.0,69.269402,2.415094,-1.0,-0.592872,-22.501357,Play,3.0,Direct,6.0,32.0,81.0,967.0,162.623030,4.103896,-6.0,-1.688802,-93.353629,0.015692


In [63]:
df_test_model.loc[(df_test_model['player_name'] == 'Alina Muller')]

,player_name,is_goal,event_type_code,x_coord,y_coord,distance_to_goal,angle_to_goal,prev_event_code,prev_event_x_coord,prev_event_y_coord,second_prev_event_code,second_prev_event_x_coord,second_prev_event_y_coord,game_seconds_remaining,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,goal_diff,strength_state,expected_goals
268,Alina Muller,0,26,179,50,13.313527,1.466667,4.0,179.0,50.0,5.0,163.0,67.0,2382.0,0.0,0.000000,0.000000,-4.0,0.364626,-23.145354,0,0,0.132416
286,Alina Muller,0,26,172,50,19.500000,2.400000,3.0,177.0,3.0,4.0,193.0,13.0,1897.0,-2.0,2.070886,-22.084252,-1.0,0.430809,11.932102,0,0,0.052110
293,Alina Muller,0,21,164,46,26.234519,7.428571,3.0,193.0,32.0,4.0,195.0,64.0,1647.0,-1.0,7.714286,15.314354,-5.0,-0.053156,-11.153575,0,0,0.142634
298,Alina Muller,0,21,146,4,58.465802,1.142857,4.0,147.0,2.0,3.0,192.0,31.0,1406.0,0.0,0.081129,-0.604072,-3.0,1.235641,47.397256,1,0,0.000895
304,Alina Muller,0,21,180,50,12.500000,1.333333,4.0,180.0,50.0,5.0,165.0,7.0,1188.0,0.0,0.000000,0.000000,-2.0,0.629108,-30.919466,0,0,0.128020
321,Alina Muller,0,26,141,26,51.703482,2.969697,3.0,132.0,42.0,3.0,150.0,5.0,737.0,-2.0,-113.030303,-6.298674,-2.0,114.933333,3.172875,1,1,0.013639
327,Alina Muller,0,19,160,27,33.767588,1.935484,3.0,162.0,63.0,3.0,134.0,38.0,642.0,-1.0,0.569630,-0.934717,-2.0,-11.078591,-21.478207,1,1,0.018059
338,Alina Muller,0,21,128,31,63.057513,5.391304,4.0,180.0,20.0,3.0,192.0,32.0,211.0,-5.0,4.946860,38.435369,-3.0,0.634921,13.933365,1,0,0.037201
343,Alina Muller,1,26,186,38,6.020797,0.888889,3.0,171.0,63.0,3.0,135.0,62.0,158.0,0.0,-0.037940,-21.930052,-2.0,-1.893684,-30.403670,0,0,0.124871
352,Alina Muller,1,21,35,21,156.484025,7.209302,4.0,7.0,58.0,5.0,192.0,14.0,5.0,-3.0,-4.597149,-27.171223,-1.0,11.876627,155.085158,1,-2,0.090520


In [64]:
df_test.loc[(df_test['player_name'] == 'Alina Muller')]

,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event
268,4183,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,2,1182,5 on 5,0,0,Alina Muller,Shot,0,179,50,Wristshot,None,None,None,Missed,0,0,NaN,None,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,1,0,2382.0,5,26,5,5,0,13.313527,1.466667,Puck Recovery,4.0,None,-1.0,179.0,50.0,2382.0,13.313527,1.466667,0.0,0.000000,0.000000,Shot,5.0,Wristshot,26.0,163.0,67.0,2386.0,36.458881,1.102041,-4.0,0.364626,-23.145354
286,4428,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,2,697,5 on 5,0,0,Alina Muller,Shot,0,172,50,Wristshot,None,None,None,On Net,0,0,NaN,None,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,2,0,1897.0,5,26,5,5,0,19.500000,2.400000,Play,3.0,Direct,6.0,177.0,3.0,1899.0,41.584252,0.329114,-2.0,2.070886,-22.084252,Puck Recovery,4.0,None,-1.0,193.0,13.0,1900.0,29.652150,-0.101695,-1.0,0.430809,11.932102
293,4551,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,2,447,5 on 5,0,0,Alina Muller,Shot,0,164,46,Snapshot,None,None,None,Missed,1,1,NaN,None,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,1,0,1647.0,5,21,5,5,0,26.234519,7.428571,Play,3.0,Direct,6.0,193.0,32.0,1648.0,10.920165,-0.285714,-1.0,7.714286,15.314354,Puck Recovery,4.0,None,-1.0,195.0,64.0,1653.0,22.073740,-0.232558,-5.0,-0.053156,-11.153575
298,4666,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,2,206,5 on 5,1,0,Alina Muller,Shot,0,146,4,Snapshot,None,None,None,Missed,1,0,NaN,None,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,1,1,1406.0,5,21,5,5,0,58.465802,1.142857,Puck Recovery,4.0,None,-1.0,147.0,2.0,1406.0,59.069874,1.061728,0.0,0.081129,-0.604072,Play,3.0,Direct,6.0,192.0,31.0,1409.0,11.672618,-0.173913,-3.0,1.235641,47.397256
304,4773,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,1188,5 on 5,1,1,Alina Muller,Shot,0,180,50,Snapshot,None,None,None,On Net,0,0,NaN,None,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,2,0,1188.0,5,21,5,5,0,12.500000,1.333333,Puck Recovery,4.0,None,-1.0,180.0,50.0,1188.0,12.500000,1.333333,0.0,0.000000,0.000000,Shot,5.0,Wristshot,26.0,165.0,7.0,1190.0,43.419466,0.704225,-2.0,0.629108,-30.919466
321,4996,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,737,5 on 4,2,1,Alina Muller,Shot,0,141,26,Wristshot,None,None,None,Blocked,1,0,219.0,None,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,0,1,737.0,5,26,5,4,1,51.703482,2.969697,Play,3.0,Direct,6.0,132.0,42.0,739.0,58.002155,116.000000,-2.0,-113.030303,-6.298674,Play,3.0,Dire

In [65]:
df_test.shape

(709, 67)

In [66]:
df_test[df_test['strength_state'] >0].shape, df_test_model[df_test_model['strength_state'] >0].shape

((124, 67), (124, 23))

## xG values for all coordinates

In [67]:
df_non_shot_train = pd.read_sql(
    '''
    SELECT is_goal, detail_1_code event_type_code, x_coord, y_coord, distance_to_goal, angle_to_goal, prev_event_code, prev_event_x_coord, prev_event_y_coord, second_prev_event_code, second_prev_event_x_coord, second_prev_event_y_coord, game_seconds_remaining, time_diff_last_event, angle_diff_last_event, distance_diff_last_event, prev_time_diff_last_event, prev_angle_diff_last_event, prev_distance_diff_last_event, goal_diff, strength_state
    FROM pbp_training 
    WHERE (is_shot = 1 OR is_goal = 1)
    '''
,conn)

In [68]:
df_non_shot_test = pd.read_sql(
    '''
    SELECT * FROM play_by_play
    '''
,conn)

In [69]:
df_non_shot_train.shape, df_non_shot_test.shape

((3656, 21), (10609, 67))

In [70]:
non_shot_features = ['x_coord', 'y_coord', 'distance_to_goal', 'angle_to_goal', 'prev_event_code', 'prev_event_x_coord', 'prev_event_y_coord', 'second_prev_event_code', 'second_prev_event_x_coord', 'second_prev_event_y_coord', 'game_seconds_remaining', 'time_diff_last_event', 'angle_diff_last_event', 'distance_diff_last_event','prev_time_diff_last_event', 'prev_angle_diff_last_event', 'prev_distance_diff_last_event', 'goal_diff', 'strength_state']
non_shot_label = ['is_goal']
X_non_shot_train = df_non_shot_train[non_shot_features]
y_non_shot_train = df_non_shot_train[non_shot_label]
X_non_shot_test = df_non_shot_test[non_shot_features]
y_non_shot_test = df_non_shot_test[non_shot_label]

In [71]:
df_non_shot_xg = pd.DataFrame()

non_shot_classifier = XGBClassifier()
non_shot_classifier = GridSearchCV(non_shot_classifier, parameters, cv=5, scoring='roc_auc', verbose=2)
non_shot_classifier.fit(X_non_shot_train, y_non_shot_train)
dfs_predictions_non_shot_xg = {}
y_non_shot_pred = non_shot_classifier.predict_proba(X_non_shot_test)
dfs_predictions_non_shot_xg[non_shot_label[0]] = pd.Series(y_non_shot_pred[:, 1],index=df_non_shot_test.index)
df_non_shot_xg = pd.concat(dfs_predictions_non_shot_xg, axis=1)

non_shot_auc_roc = roc_auc_score(y_non_shot_test, y_non_shot_pred[:, 1])
non_shot_auc_roc

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[14:54:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, nthread=4, objective=binary:logistic, seed=42; total time=   0.2s
[14:54:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, nthread=4, objective=binary:logistic, seed=42; total time=   0.1s
[14:54:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, nthread=4, objective=binary:logistic, seed=42; total time=   0.1s
[14:54:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, nthread=4, objective=binary:logistic, seed=42; total time=   0.1s
[14:54:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, nthread=4, objective=binary:logistic, seed=42; total time=   0.2s
[14:54:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=6, n_estimators=1000, nthread=4, objective=binary:logistic, seed=42; total time=   2.3s
[14:55:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=6, n_estimators=1000, nthread=4, objective=binary:logistic, seed=42; total time=   2.5s
[14:55:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.38693993995489145

In [72]:
df_non_shot_test['non_shot_expected_goals'] = df_non_shot_xg['is_goal']

In [73]:
df_non_shot_model = df_non_shot_test.merge(df_shots[['index','expected_goals']], on='index', how='left')

In [74]:
df_non_shot_model.loc[df_non_shot_model['is_shot']==1,['player_name','index','expected_goals','non_shot_expected_goals']].head()

,player_name,index,expected_goals,non_shot_expected_goals
12,Jocelyne Larocque,12,0.007298,0.189897
18,Sarah Fillier,18,0.001768,0.189897
58,Abby Roque,58,0.043121,0.198701
68,Kelly Pannek,68,0.020900,0.189897
82,Sarah Nurse,82,0.006565,0.189897


In [75]:
df_non_shot_model.to_sql('play_by_play', conn, if_exists='replace')

10609

In [76]:
df_non_shot_model.loc[(df_non_shot_model['venue'] == 'home') & (df_non_shot_model['period'] == 2),['expected_goals','non_shot_expected_goals']].describe()

,expected_goals,non_shot_expected_goals
count,136.000000,1934.000000
mean,0.049273,0.272831
std,0.063209,0.070751
min,0.000815,0.189897
25%,0.011382,0.197509
50%,0.022517,0.317673
75%,0.067483,0.328330
max,0.437306,0.636790


In [77]:
df_non_shot_model.loc[(df_non_shot_model['venue'] == 'away') & (df_non_shot_model['period'] == 2),['expected_goals','non_shot_expected_goals']].describe()

,expected_goals,non_shot_expected_goals
count,116.000000,1641.000000
mean,0.042892,0.279183
std,0.054105,0.071311
min,0.000895,0.189897
25%,0.009961,0.197509
50%,0.020653,0.328330
75%,0.057372,0.328330
max,0.272564,0.626274


In [78]:
df_non_shot_model.loc[(df_non_shot_model['venue'] == 'home') & (df_non_shot_model['period'] == 1 | 3 ),['expected_goals','non_shot_expected_goals']].describe()

,expected_goals,non_shot_expected_goals
count,137.000000,1864.000000
mean,0.040523,0.266966
std,0.044224,0.072241
min,0.001789,0.189897
25%,0.011756,0.189897
50%,0.022631,0.260191
75%,0.057516,0.328330
max,0.293761,0.472939


In [79]:
df_non_shot_model.loc[(df_non_shot_model['venue'] == 'away') & (df_non_shot_model['period'] == 3 | 1),['expected_goals','non_shot_expected_goals']].describe()

,expected_goals,non_shot_expected_goals
count,91.000000,1532.000000
mean,0.037897,0.278033
std,0.045029,0.070877
min,0.003646,0.189897
25%,0.009727,0.197509
50%,0.016342,0.317673
75%,0.048769,0.328330
max,0.201734,0.608897


In [80]:
df_non_shot_model.head()

,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,non_shot_expected_goals,expected_goals
0,0,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1200,5 on 5,0,0,Marie-Philip Poulin,Faceoff Win,1,100,42,Backhand,Hannah Brandt,NaN,NaN,None,0,0,NaN,NaN,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,1,0,60,-1,0,3600.0,1,0,5,5,0,90.001389,180.000000,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.328330,NaN
1,1,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1199,5 on 5,0,0,Jocelyne Larocque,Puck Recovery,1,86,31,None,None,NaN,NaN,None,0,0,NaN,NaN,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,4,0,42,-1,0,3599.0,4,-1,5,5,0,104.633886,9.043478,Faceoff Win,1.0,Backhand,0.0,100.0,42.0,3600.0,90.001389,180.000000,-1.0,-170.956522,14.632497,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.328330,NaN
2,2,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1198,5 on 5,0,0,Jocelyne Larocque,Dump In/Out,0,96,15,None,None,NaN,NaN,None,0,0,NaN,NaN,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,0,0,42,-1,0,3598.0,0,-1,5,5,0,97.940033,3.418182,Puck Recovery,4.0,None,-1.0,86.0,31.0,3599.0,104.633886,9.043478,-1.0,-5.625296,-6.693853,Faceoff Win,1.0,Backhand,0.0,100.0,42.0,3600.0,90.001389,180.000000,-1.0,-170.956522,14.632497,0.328330,NaN
3,3,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1197,5 on 5,0,0,Jocelyne Larocque,Zone Entry,1,124,1,Dumped,Cayla Barnes,NaN,NaN,None,0,0,NaN,NaN,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,7,0,42,-1,0,3597.0,7,7,5,5,0,77.963132,1.590361,Dump In/Out,0.0,None,-1.0,96.0,15.0,3598.0,97.940033,3.418182,-1.0,-1.827820,-19.976900,Puck Recovery,4.0,None,-1.0,86.0,31.0,3599.0,104.633886,9.043478,-1.0,-5.625296,-6.693853,0.328330,NaN
4,4,8/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Canada,home,1,1194,5 on 5,0,0,Lee Stecklein,Puck Recovery,1,6,73,None,None,NaN,NaN,None,0,0,NaN,NaN,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,4,4,53,-1,0,3594.0,4,-1,5,5,0,186.510724,6.032787,Zone Entry,7.0,Dumped,7.0,124.0,1.0,3597.0,77.963132,1.590361,-3.0,4.442425,108.547591,Dump In/Out,0.0,None,-1.0,96.0,15.0,3598.0,97.940033,3.418182,-1.0,-1.827820,-19.976900,0.197509,NaN


## xG values for NWHL pbp shots 

In [81]:
features_nwhl = ['x_coord', 'y_coord', 'distance_to_goal', 'angle_to_goal', 'prev_event_code', 'prev_event_x_coord', 'prev_event_y_coord', 'second_prev_event_code', 'second_prev_event_x_coord', 'second_prev_event_y_coord', 'game_seconds_remaining', 'time_diff_last_event', 'angle_diff_last_event', 'distance_diff_last_event','prev_time_diff_last_event', 'prev_angle_diff_last_event', 'prev_distance_diff_last_event', 'goal_diff', 'strength_state']
label = ['is_goal']

In [82]:
df_nwhl_shots = pd.read_sql(
    '''
    Select player_name, team_name, is_goal, x_coord, y_coord, distance_to_goal, angle_to_goal, prev_event_code, prev_event_x_coord, prev_event_y_coord, second_prev_event_code, second_prev_event_x_coord, second_prev_event_y_coord, game_seconds_remaining, time_diff_last_event, angle_diff_last_event, distance_diff_last_event, prev_time_diff_last_event, prev_angle_diff_last_event, prev_distance_diff_last_event, goal_diff, strength_state 
    FROM pbp_nwhl 
    WHERE event = 'Shot' OR event = 'Goal';
    ''',conn
).dropna()

In [83]:
df_nwhl_shots.shape

(5141, 22)

In [84]:
df_train_model = pd.read_sql(
    '''
    SELECT is_goal, x_coord, y_coord, distance_to_goal, angle_to_goal, prev_event_code, prev_event_x_coord, prev_event_y_coord, second_prev_event_code, second_prev_event_x_coord, second_prev_event_y_coord, game_seconds_remaining, time_diff_last_event, angle_diff_last_event, distance_diff_last_event, prev_time_diff_last_event, prev_angle_diff_last_event, prev_distance_diff_last_event, goal_diff, strength_state
    FROM pbp_training 
    WHERE (is_shot = 1 OR is_goal = 1)
    '''
,conn)

In [85]:
X_train

,event_type_code,x_coord,y_coord,distance_to_goal,angle_to_goal,prev_event_code,prev_event_x_coord,prev_event_y_coord,second_prev_event_code,second_prev_event_x_coord,second_prev_event_y_coord,game_seconds_remaining,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,goal_diff,strength_state
0,30,154,36,36.582099,5.538462,9.0,125.0,31.0,5.0,113.0,84.0,1158.0,0.0,-0.113712,-29.427370,-1.0,3.796752,-21.461955,0,0.0
1,30,177,39,13.462912,3.714286,5.0,163.0,70.0,9.0,125.0,79.0,1144.0,-1.0,2.732468,-25.076029,-1.0,-0.799004,-36.008024,0,0.0
2,12,128,82,73.513604,1.569620,5.0,163.0,84.0,9.0,124.0,70.0,1133.0,-1.0,0.919018,24.003504,-2.0,-1.749398,-21.989900,0,1.0
3,28,152,67,45.213383,1.551020,5.0,130.0,28.0,5.0,140.0,4.0,1095.0,-2.0,-2.586911,-16.513840,-2.0,2.839230,-1.377849,0,1.0
4,30,174,58,22.276669,1.032258,6.0,171.0,59.0,1.0,169.0,64.0,1094.0,0.0,-0.119257,-2.887790,-1.0,0.174771,-4.889659,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4290,26,135,41,55.020451,36.666667,4.0,135.0,22.0,1.0,169.0,20.0,269.0,-1.0,33.983740,-3.675801,-1.0,1.749593,27.918825,3,0.0
4291,21,187,47,5.408327,0.666667,3.0,158.0,20.0,4.0,158.0,20.0,261.0,-4.0,-0.755556,-33.710083,0.0,0.000000,0.000000,3,0.0
4292,21,175,27,21.569655,0.967742,4.0,169.0,19.0,3.0,183.0,7.0,229.0,-1.0,0.074125,-9.946214,-2.0,0.696434,-4.667691,3,0.0
4293,21,147,37,43.350317,7.818182,3.0,158.0,28.0,3.0,142.0,41.0,131.0,-1.0,5.611285,8.218423,-6.0,-29.793103,-12.891537,3,1.0


In [86]:
df_xg_model_nwhl = pd.DataFrame()
auc_roc = []
kf = KFold(10, shuffle=True)
vals_y_test = pd.DataFrame()
vals_y_pred = np.ndarray(shape=(0,))

for train_idx, text_idx in kf.split(df_nwhl_shots):
    df_train_data = df_train_model.append(df_nwhl_shots.iloc[train_idx].copy(), ignore_index=True) 
    df_test_data = df_nwhl_shots.iloc[text_idx].copy()
    X_train = df_train_data[features_nwhl]
    y_train = df_train_data[label]
    X_test = df_test_data[features_nwhl]
    y_test = df_test_data[label]

    classifier = XGBClassifier()
    classifier = GridSearchCV(classifier, parameters, cv=5, scoring='roc_auc', verbose=2)
    classifier.fit(X_train, y_train)
    dfs_predictions = {}
    y_pred = classifier.predict_proba(X_test)
    dfs_predictions[label[0]] = pd.Series(y_pred[:, 1],index=df_test_data.index)
    df_predictions = pd.concat(dfs_predictions, axis=1)
    df_xg_model_nwhl = df_xg_model_nwhl.append(df_predictions)

    vals_y_test = vals_y_test.append(y_test) 
    vals_y_pred = np.concatenate([vals_y_pred,y_pred[:, 1]], axis=0)

<ipython-input-86-fc432f7da7a0>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train_data = df_train_model.append(df_nwhl_shots.iloc[train_idx].copy(), ignore_index=True)
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-p

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[14:55:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, nthread=4, objective=binary:logistic, seed=42; total time=   0.2s
[14:55:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

...

/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=6, n_estimators=1000, nthread=4, objective=binary:logistic, seed=42; total time=   5.4s
[15:18:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/

[CV] END learning_rate=0.01, max_depth=6, n_estimators=1000, nthread=4, objective=binary:logistic, seed=42; total time=   5.2s
[15:18:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


<ipython-input-86-fc432f7da7a0>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_xg_model_nwhl = df_xg_model_nwhl.append(df_predictions)
<ipython-input-86-fc432f7da7a0>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vals_y_test = vals_y_test.append(y_test)


In [87]:
df_nwhl_shots['expected_goals'] = df_xg_model_nwhl['is_goal']

In [88]:
df_nwhl_shots[['is_goal', 'expected_goals']].describe()

,is_goal,expected_goals
count,5141.000000,5141.000000
mean,0.100564,0.098844
std,0.300780,0.073261
min,0.000000,0.018297
25%,0.000000,0.048270
50%,0.000000,0.074963
75%,0.000000,0.122751
max,1.000000,0.706037


In [89]:
new_auc_roc = roc_auc_score(vals_y_test, vals_y_pred)
new_auc_roc

0.7249756128984761

In [90]:
df_nwhl_shots.head()

,player_name,team_name,is_goal,x_coord,y_coord,distance_to_goal,angle_to_goal,prev_event_code,prev_event_x_coord,prev_event_y_coord,second_prev_event_code,second_prev_event_x_coord,second_prev_event_y_coord,game_seconds_remaining,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,goal_diff,strength_state,expected_goals
0,Devon Skeats,BUF,0.0,-66.6270,1.2750,92.402651,2.181818,3.0,-70.6662,-21.5220,1.0,0.0594,0.8245,3509.0,-53.0,-1.858998,-9.583867,-38.0,-175.959184,11.985130,0.0,0.0,0.092597
1,Jordyn Burns,BUF,0.0,57.8358,1.4960,46.970736,0.530120,1.0,-62.1720,-19.7540,7.0,-46.4112,-20.4595,3471.0,-17.0,0.000000,0.000000,0.0,-1.651698,-45.431916,0.0,0.0,0.065444
2,Emily Field,BOS,0.0,63.8946,-0.2720,187.238484,8.651163,7.0,57.8358,1.4960,1.0,-62.1720,-19.7540,3463.0,-8.0,8.121042,140.267749,-17.0,0.000000,0.000000,0.0,0.0,0.055138
3,Gigi Marvin,BOS,0.0,55.4202,-21.1735,187.238484,8.651163,7.0,63.8946,-0.2720,7.0,57.8358,1.4960,3405.0,-58.0,0.000000,0.000000,-8.0,8.121042,140.267749,0.0,0.0,0.031506
4,Corinne Buie,BUF,0.0,-64.6074,-17.6290,175.625311,4.845070,7.0,55.4202,-21.1735,7.0,63.8946,-0.2720,3403.0,-2.0,-3.806092,-11.613173,-58.0,0.000000,0.000000,0.0,0.0,0.027422


In [91]:
df_nwhl_shots.to_sql('nwhl_shots', conn, if_exists='replace')

5141